In [54]:
import pandas as pd

nutritional_values = pd.read_csv("../data/nutritional_values_macros.csv")
recipes = pd.read_csv("../data/filtered_recipes_100.csv", sep="|")
ingredients = pd.read_csv("../data/ingredients_100.csv")



recipes_nutritional_values = pd.merge(recipes,
                                         nutritional_values,
                                         on='Recipe_id', 
                                         how='right')



In [55]:
recipes_nutritional_values.head()

,Recipe_id,Categoria,Nombre,Valoracion,Dificultad,Num_comensales,Tiempo,Tipo,Link_receta,Num_comentarios,...,magnesio,sodio,fósforo,ioduro,"selenio, total",zinc (cinc),Total_Grams,grasa_cals_pctg,proteina_cals_pctg,carbos_cals_pctg
0,1,Cócteles y bebidas,Absolut tequila,3.9,baja,1.0,15m,NaN,https://www.recetasgratis.net/receta-de-absolu...,1,...,0.002320,0.004270,0.003190,0.000001,0.000000,0.000000,42.00000,4.6,7.4,88.0
1,5,Recetas de Verduras,Receta de aguacate y plátano,3.6,NaN,NaN,NaN,NaN,https://www.recetasgratis.net/receta-de-aguaca...,4,...,0.031209,0.008544,0.040281,0.000003,0.000001,0.000351,58.33445,51.2,5.7,43.1
2,8,Cócteles y bebidas,Cóctel refrescante de menta,2.3,baja,1.0,15m,NaN,https://www.recetasgratis.net/receta-de-coctel...,0,...,0.039056,0.009669,0.051344,0.000003,0.000001,0.000601,210.62500,7.6,20.9,71.5
3,10,Cócteles y bebidas,Batido de yogur y plátanos,3.7,NaN,NaN,NaN,NaN,https://www.recetasgratis.net/receta-de-batido...,0,...,0.197200,0.394500,0.744000,0.000042,0.000011,0.003215,1100.00000,4.8,20.6,74.5
4,27,Recetas de Pescado,Receta de aperitivo de atún,4.0,NaN,NaN,NaN,NaN,https://www.recetasgratis.net/receta-de-aperit...,4,...,0.045692,1.049754,0.180909,0.000010,0.000031,0.001117,404.92792,82.8,3.0,14.2


In [59]:
def get_food_percentage(tdee):
    
    breakfast = tdee * 0.2
    snack_1   = tdee * 0.1
    lunch     = tdee * 0.35
    snack_2   = tdee * 0.1
    dinner    = tdee * 0.25
    
    return breakfast, snack_1, lunch, snack_2, dinner

DINNER_CAT = ['Cena']
LUNCH_CAT = ['Plato principal','Entrante','Acompañamiento']
SNACK_CAT = ['Merienda','Cumpleaños','Postre']
BREAKFAST_CAT = ['Desayuno']

categories={'Desayuno': BREAKFAST_CAT,
                    'Comida': LUNCH_CAT,
                    'Snack': SNACK_CAT,
                    'Cena': DINNER_CAT
                    }

 #RANGO DE PORCENTAJES DE CARBOS,PROTEÍNAS Y GRASAS
    
GAIN = [40,60,25,35,15,25]
MAINTAIN = [30,50,25,35,25,35]
LOSS = [10,30,40,50,30,40]
    
objetivos={
    0 : GAIN,
    1 : MAINTAIN,
    2 : LOSS
        }

def get_food(category, calories,pctg_array):
    
    min_carbos = pctg_array[0]
    max_carbos = pctg_array[1]
    min_protes = pctg_array[2]
    max_protes = pctg_array[3]
    min_grasas = pctg_array[4]
    max_grasas = pctg_array[5]
        
    selected_category = recipes_nutritional_values[(recipes_nutritional_values['Tipo'].isin(categories[category])) 
                                    & (recipes_nutritional_values["energia"] > (calories*0.9) )
                                     & (recipes_nutritional_values["energia"] < (calories * 1.1))]
    
    accomplish_macros = selected_category[(selected_category["grasa_cals_pctg"] > min_grasas)
                                          & (selected_category["grasa_cals_pctg"] < max_grasas)
                                          & (selected_category["proteina_cals_pctg"] > min_protes) 
                                          & (selected_category["proteina_cals_pctg"] < max_protes)
                                          & (selected_category["carbos_cals_pctg"] > min_carbos) 
                                          & (selected_category["carbos_cals_pctg"] < max_carbos)]
    
    print(accomplish_macros.shape[0])
    
    selected_recipe = selected_category[['Recipe_id', 'Nombre', 'energia']].sample(n=1)
    
    recipe_name = selected_recipe.iloc[0]['Nombre']
    recipe_id = selected_recipe.iloc[0]['Recipe_id']
    recipe_calories = selected_recipe.iloc[0]['energia']
    
    recipe = {
        'Nombre':selected_recipe.iloc[0]['Nombre'],
        'Id': selected_recipe.iloc[0]['Recipe_id'],
        'Calorias': selected_recipe.iloc[0]['energia']} 
    
    return recipe_id, recipe_name, recipe_calories


def get_macros(objective):

    pctg_array = objetivos.get(objective)  
    
    return pctg_array
    


def get_recommendations(tdee, pctg_array):
    
    breakfast_cal, snack_1_cal, lunch_cal, snack_2_cal, dinner_cal = get_food_percentage(tdee)
    
    breakfast = get_food('Desayuno',breakfast_cal,pctg_array)
    
    snack_1 = get_food('Snack',snack_1_cal, pctg_array)

    lunch = get_food('Comida',lunch_cal, pctg_array)

    snack_2 = get_food('Snack',snack_2_cal, pctg_array)

    dinner = get_food('Cena', dinner_cal, pctg_array)
                                                                                                                            
                                                            
    print("*** PLAN DE COMIDAS ***")
    print("DESAYUNO: ", breakfast)
    print("SNACK_1: ", snack_1 )                                                                                                                        
    print("COMIDA: ", lunch )
    print("SNACK_2: ", snack_2 )                                                                                                                        
    print("CENA: ", dinner )
    
    return breakfast, snack_1, lunch, snack_2, dinner                                                                                                               
                                                                                                                            
                                                                                                                 
                   

In [60]:
tdee_sample = 2500
objective = 1 # mantain
pctg_array = get_macros(objective)


In [61]:
breakfast, snack_1, lunch, snack_2, dinner = get_recommendations(tdee_sample,pctg_array)

TypeError: Cannot perform 'rand_' with a dtyped [float64] array and scalar of type [bool]